In [2]:
import os
os.getcwd() 
os.chdir('/Users/hannahz/Desktop/G5055_Practicum_Project2/Data/Text_Model_Data')

In [3]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hannahz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_excel('definition_filled.xlsx')

In [5]:
#remove \n
df['definition'] = df['definition'].replace(r'\n',' ', regex=True) 
#remove multiple whitespace
df['definition'] = df['definition'].replace('\s+', ' ', regex=True)
#remove white space in the beginning and end
df['definition'] = df['definition'].str.strip()
stop_words_l=stopwords.words('english')
# removing special characters and stop words from the text and lower case (age) - age,  (the - ''
df['definition']=df['definition'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]','',w).lower() not in stop_words_l) )

#remove white space 
df['definition'] = df['definition'].replace('\s+', ' ', regex=True)

In [6]:
#stem
stemmer = SnowballStemmer('english')
stemmer.stem('indicator')
df['definition'] = [" ".join([stemmer.stem(word) for word in sentence.split(" ")]) for sentence in df['definition']]

In [7]:
df['definition']

0      indic proport popul intern poverti line defin ...
1      nation poverti rate percentag total popul live...
2      follow four seri use monitor sdg offici multid...
3      indic reflect proport person effect cover soci...
4      follow key concept defin support indic context...
                             ...                        
241    indic base wbg data amount unit state dollar c...
242    indic refer number countri nation statist legi...
243    indic number countri nation statist plan fulli...
244    indic dollar valu resourc made avail strengthe...
245    indic track proport countri conduct least one ...
Name: definition, Length: 246, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(df['definition'])
tfidf_vectors=tfidfvectoriser.transform(df['definition'])

#calculate similarity based on the vectors
pairwise_similarities=cosine_similarity(tfidf_vectors.toarray())

In [9]:
def similardocs_one(doc_id,similarity_matrix):
    #find the index for the doc_id
    index = df.iloc[doc_id]['Index']
    #initiated related index
    related_index = []
    similar_score = similarity_matrix[doc_id]
    score_sort = np.sort(similar_score)[::-1]
    #find related doc_id, sort from most similar to least similar 
    similar_ix=np.argsort(similarity_matrix[doc_id])[::-1] # sort doc_id from most similar to least 
    #find corresponded index for doc_id
    for ix in similar_ix:
        if ix==doc_id:
            continue
        related_index.append(df.iloc[ix]["Index"])
    #create a dataframe of
    relation_df = pd.DataFrame({'indicator': np.repeat(index, len(related_index)),
                                'related_indicator': related_index,
                                'similarity_score': score_sort[1:]})
    return relation_df

list_of_dataframes = [similardocs_one(x,pairwise_similarities) for x in range(len(df['Index']))]
outcome = pd.concat(list_of_dataframes)

In [10]:
outcome.shape

(60270, 3)

In [11]:
#choose the threshold of 0.2 and keep all the duplicate pairs
outcome_pair = outcome[outcome['similarity_score'] >= 0.2]
outcome_pair

,indicator,related_indicator,similarity_score
0,1.1.1,1.2.1,0.486675
0,1.2.1,1.1.1,0.486675
0,1.2.2,1.b.1,0.212234
0,1.3.1,17.1.1,0.236192
0,1.4.1,6.2.1,0.375773
...,...,...,...
2,17.18.3,17.18.2,0.245164
3,17.18.3,17.5.1,0.240845
0,17.19.1,17.18.3,0.308272
1,17.19.1,17.18.2,0.228812


In [12]:
most_central_indicator = outcome_pair.groupby(['indicator'])['related_indicator'].count().reset_index(name = 'num_of_relatedindic').sort_values(by = 'num_of_relatedindic',ascending=False)

In [13]:
most_central_indicator = most_central_indicator.reset_index(drop = True)

In [14]:
most_central_indicator

,indicator,num_of_relatedindic
0,17.2.1,14
1,1.a.1,11
2,1.a.2,11
3,5.3.1,10
4,16.2.3,10
...,...,...
191,16.7.2,1
192,16.5.2,1
193,16.4.1,1
194,16.3.3,1


In [19]:
#save it to data 
#most_central_indicator.to_csv('most_central_indicator.csv')

In [23]:
ind17 = outcome_pair[outcome_pair['indicator'] == '17.2.1']
ind17 

,indicator,related_indicator,similarity_score
0,17.2.1,8.a.1,0.396998
1,17.2.1,1.a.1,0.377151
2,17.2.1,6.a.1,0.330725
3,17.2.1,17.12.1,0.298103
4,17.2.1,2.a.2,0.281884
5,17.2.1,17.9.1,0.272766
6,17.2.1,10.b.1,0.270509
7,17.2.1,9.a.1,0.264087
8,17.2.1,4.b.1,0.263449
9,17.2.1,10.a.1,0.244776


In [26]:
ind17.to_csv('ind17.csv', index=False)

In [27]:
pd.read_csv('ind17.csv')

,indicator,related_indicator,similarity_score
0,17.2.1,8.a.1,0.396998
1,17.2.1,1.a.1,0.377151
2,17.2.1,6.a.1,0.330725
3,17.2.1,17.12.1,0.298103
4,17.2.1,2.a.2,0.281884
5,17.2.1,17.9.1,0.272766
6,17.2.1,10.b.1,0.270509
7,17.2.1,9.a.1,0.264087
8,17.2.1,4.b.1,0.263449
9,17.2.1,10.a.1,0.244776


In [45]:
#drop duplicate pair
outcome = outcome.reset_index(drop = True)
outcome_new = outcome.loc[pd.DataFrame(np.sort(outcome[['indicator', 'related_indicator']],1),index=outcome.index).drop_duplicates(keep='first').index]

In [46]:
outcome_new

,indicator,related_indicator,similarity_score
0,1.1.1,1.2.1,0.486675
1,1.1.1,10.2.1,0.196380
2,1.1.1,2.c.1,0.195556
3,1.1.1,16.b.1,0.184231
4,1.1.1,10.3.1,0.184231
...,...,...,...
59291,17.18.2,17.19.1,0.228812
59297,17.18.2,17.19.2,0.109456
59535,17.18.3,17.19.1,0.308272
59557,17.18.3,17.19.2,0.116123


In [49]:
outcome_nopair = outcome_new[outcome_new['similarity_score'] >= 0.2]
outcome_nopair = outcome_nopair.reset_index(drop = True)

In [50]:
outcome_nopair

,indicator,related_indicator,similarity_score
0,1.1.1,1.2.1,0.486675
1,1.2.2,1.b.1,0.212234
2,1.3.1,17.1.1,0.236192
3,1.4.1,6.2.1,0.375773
4,1.4.1,11.6.1,0.363539
...,...,...,...
347,17.11.1,17.12.1,0.327914
348,17.15.1,17.16.1,0.210638
349,17.18.2,17.18.3,0.245164
350,17.18.2,17.19.1,0.228812


In [53]:
outcome_nopair.to_csv('edgelist.csv',index = False)

In [57]:
pd.read_csv('edgelist.csv')

,indicator,related_indicator,similarity_score
0,1.1.1,1.2.1,0.486675
1,1.2.2,1.b.1,0.212234
2,1.3.1,17.1.1,0.236192
3,1.4.1,6.2.1,0.375773
4,1.4.1,11.6.1,0.363539
...,...,...,...
347,17.11.1,17.12.1,0.327914
348,17.15.1,17.16.1,0.210638
349,17.18.2,17.18.3,0.245164
350,17.18.2,17.19.1,0.228812


In [58]:
pd.read_csv('nodes_246.csv')

,Index
0,1.1.1
1,1.2.1
2,1.2.2
3,1.3.1
4,1.4.1
...,...
241,17.17.1
242,17.18.2
243,17.18.3
244,17.19.1


In [56]:
outcome_nopair.groupby(['indicator'])['related_indicator'].count()

indicator
1.1.1    1
1.2.2    1
1.3.1    1
1.4.1    8
1.4.2    8
        ..
9.4.1    1
9.5.1    3
9.5.2    2
9.a.1    3
9.b.1    1
Name: related_indicator, Length: 141, dtype: int64

In [125]:
top2[top2['indicator'] == '10.2.1']

,indicator,related_indicator,similarity_score
0,10.2.1,10.1.1,0.251813
1,10.2.1,7.2.1,0.221586


In [124]:
top2[(top2['indicator'] == '1.1.1') | (top2['related_indicator'] == '1.1.1')]

,indicator,related_indicator,similarity_score
0,1.1.1,1.2.1,0.486675
1,1.1.1,10.2.1,0.196380
0,1.2.1,1.1.1,0.486675
1,2.c.1,1.1.1,0.195556
0,11.6.2,1.1.1,0.122446


In [110]:
goal_num = top2['indicator']
top2['ind_goal'] = [re.findall(r'^(\d+).', goal)[0] for goal in goal_num]

<ipython-input-110-9ee316a44c80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2['ind_goal'] = [re.findall(r'^(\d+).', goal)[0] for goal in goal_num]


In [112]:
relate_num = top2['related_indicator']
top2['rel_goal'] = [re.findall(r'^(\d+).', goal)[0] for goal in relate_num]

<ipython-input-112-7e470b81ed0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2['rel_goal'] = [re.findall(r'^(\d+).', goal)[0] for goal in relate_num]


In [113]:
top2 

,indicator,related_indicator,similarity_score,ind_goal,rel_goal
0,1.1.1,1.2.1,0.486675,1,1
1,1.1.1,10.2.1,0.196380,1,10
0,1.2.1,1.1.1,0.486675,1,1
1,1.2.1,9.1.1,0.183131,1,9
0,1.2.2,1.b.1,0.212234,1,1
1,1.2.2,17.10.1,0.190014,1,17
0,1.3.1,17.1.1,0.236192,1,17
1,1.3.1,14.5.1,0.176815,1,14
0,1.4.1,6.2.1,0.375773,1,6
1,1.4.1,11.6.1,0.363539,1,11


In [54]:
my_vec = pd.DataFrame(tfidf_vectors.toarray())
my_vec.columns = tfidfvectoriser.vocabulary_

In [58]:
my_vec.iloc[:, -20:-1]

,participatori,concern,crfs,wbg,ppps,contractu,transmiss,pipelin,backbon,airport,port,potabl,sewerag,behind,constraint,unfop,nsds,collabor,await
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.014637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017259,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
re.sub(r'[^a-zA-Z]','', 'age is my')

'ageismy'

In [38]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/hannahz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
# get the data into the format neede for doc2vec (tagged data )
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df['definition'])]

In [40]:
model = Doc2Vec(vector_size = 300, alpha = 0.025,min_count=5,dm =1,epochs = 50) #alpha learning rate, 
model.build_vocab(tagged_data)

#train the model
model.train(tagged_data,total_examples=model.corpus_count,# number of documents 246
                        epochs=model.epochs)

#gain the word embeddings from the model
document_embeddings=np.zeros((246,300))
for i in range(len(document_embeddings)):
    document_embeddings[i]= model.docvecs[i]

<ipython-input-40-3938c60db6a3>:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings[i]= model.docvecs[i]


In [41]:
pairwise_similarities=cosine_similarity(document_embeddings)

list_of_dataframes = [similardocs_one(x,pairwise_similarities) for x in range(len(df['Index']))]
outcome = pd.concat(list_of_dataframes)

In [44]:
outcome[outcome['indicator'] == '3.1.2']

,indicator,related_indicator,similarity_score
0,3.1.2,4.2.1,0.881529
1,3.1.2,5.b.1,0.853677
2,3.1.2,16.2.3,0.803957
3,3.1.2,5.3.1,0.795917
4,3.1.2,16.9.1,0.780625
...,...,...,...
240,3.1.2,14.4.1,-0.023376
241,3.1.2,2.4.1,-0.053847
242,3.1.2,3.3.3,-0.062923
243,3.1.2,15.3.1,-0.076476


In [82]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hannahz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [83]:
#lower case
df['definition_test'] = df['definition']
df['definition_test'] = df['definition_test'].apply(lambda x: x.lower())

# remove tags
df['definition_test'] = df['definition_test'].apply(lambda x: re.sub('<[^<]+?>','', x))

# remove number
df['definition_test'] = df['definition_test'].apply(lambda x: ''.join(c for c in x if not c.isdigit()))

# Remove punctuation
df['definition_test'] = df['definition_test'].apply(lambda x: ''.join(c for c in x if not c in punctuation))

# Stop words removal
stopword = stopwords.words('english')
df['definition_test'] = df['definition_test'].apply(lambda x: ' '.join(c for c in nltk.word_tokenize(x) if not c in stopword))

# Lemmatize
stopword = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
df['definition_test'] = df['definition_test'].apply(lambda x: ' '.join(c for c in [wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))

# Stemming
snowball_stemmer = SnowballStemmer('english')
df['definition_test'] = df['definition_test'].apply(lambda x: ' '.join(c for c in [snowball_stemmer.stem(word) for word in nltk.word_tokenize(x)]))

In [84]:
df['definition_test']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(df['definition_test'])
tfidf_vectors=tfidfvectoriser.transform(df['definition_test'])

#calculate similarity based on the vectors
pairwise_similarities=cosine_similarity(tfidf_vectors.toarray())

In [86]:
def similardocs_one(doc_id,similarity_matrix):
    #find the index for the doc_id
    index = df.iloc[doc_id]['Index']
    #initiated related index
    related_index = []
    similar_score = similarity_matrix[doc_id]
    score_sort = np.sort(similar_score)[::-1]
    #find related doc_id, sort from most similar to least similar 
    similar_ix=np.argsort(similarity_matrix[doc_id])[::-1] # sort doc_id from most similar to least 
    #find corresponded index for doc_id
    for ix in similar_ix:
        if ix==doc_id:
            continue
        related_index.append(df.iloc[ix]["Index"])
    #create a dataframe of
    relation_df = pd.DataFrame({'indicator': np.repeat(index, len(related_index)),
                                'related_indicator': related_index,
                                'similarity_score': score_sort[1:]})
    return relation_df

list_of_dataframes = [similardocs_one(x,pairwise_similarities) for x in range(len(df['Index']))]
outcome = pd.concat(list_of_dataframes)

In [97]:
outcome[outcome['indicator'] == '4.4.1']

,indicator,related_indicator,similarity_score
0,4.4.1,17.8.1,0.197093
1,4.4.1,4.3.1,0.181967
2,4.4.1,8.6.1,0.157531
3,4.4.1,8.10.2,0.152086
4,4.4.1,4.6.1,0.129695
5,4.4.1,2.5.2,0.122325
6,4.4.1,5.b.1,0.111159
7,4.4.1,14.a.1,0.110816
8,4.4.1,17.7.1,0.097278
9,4.4.1,17.17.1,0.092870


In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/hannahz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# get the data into the format neede for doc2vec (tagged data )
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df['definition'])]

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
model = Doc2Vec(vector_size = 300, alpha = 0.025,min_count=5,dm =1,epochs = 50) #alpha learning rate, 
model.build_vocab(tagged_data)

#train the model
model.train(tagged_data,total_examples=model.corpus_count,# number of documents 246
                        epochs=model.epochs)

#gain the word embeddings from the model
document_embeddings=np.zeros((246,300))
for i in range(len(document_embeddings)):
    document_embeddings[i]= model.docvecs[i]

<ipython-input-14-c4fa26c25cb8>:12: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings[i]= model.docvecs[i]


In [15]:
def similardocs_one(doc_id,similarity_matrix):
    #find the index for the doc_id
    index = df.iloc[doc_id]['Index']
    #initiated related index
    related_index = []
    similar_score = similarity_matrix[doc_id]
    score_sort = np.sort(similar_score)[::-1]
    #find related doc_id, sort from most similar to least similar 
    similar_ix=np.argsort(similarity_matrix[doc_id])[::-1] # sort doc_id from most similar to least 
    #find corresponded index for doc_id
    for ix in similar_ix:
        if ix==doc_id:
            continue
        related_index.append(df.iloc[ix]["Index"])
    #create a dataframe of
    relation_df = pd.DataFrame({'indicator': np.repeat(index, len(related_index)),
                                'related_indicator': related_index,
                                'similarity_score': score_sort[1:]})
    return relation_df

list_of_dataframes = [similardocs_one(x,pairwise_similarities) for x in range(len(df['Index']))]
outcome = pd.concat(list_of_dataframes)

In [16]:
pairwise_similarities=cosine_similarity(document_embeddings)

list_of_dataframes = [similardocs_one(x,pairwise_similarities) for x in range(len(df['Index']))]
outcome = pd.concat(list_of_dataframes)

In [19]:
outcome[outcome['indicator'] == '2.1.1']

,indicator,related_indicator,similarity_score
0,2.1.1,12.a.1,0.812082
1,2.1.1,7.b.1,0.795122
2,2.1.1,7.2.1,0.786445
3,2.1.1,3.5.1,0.739189
4,2.1.1,3.a.1,0.738176
...,...,...,...
240,2.1.1,16.a.1,-0.082343
241,2.1.1,13.3.1,-0.084153
242,2.1.1,12.8.1,-0.098086
243,2.1.1,17.11.1,-0.116388
